In [3]:
import ipywidgets as widgets
from IPython.display import display

In [16]:
widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Slider:',
    value=3
)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
display(start_date_picker)

slider = widgets.IntSlider(
    min=0,
    max=9,
    step=1,
    readout=False, 
)
btn_left = widgets.Button(icon="chevron-left")
btn_right = widgets.Button(icon="chevron-right")
box = widgets.Box([btn_left, slider, btn_right])
display(box)

def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
btn.on_click(btn_eventhandler)

DatePicker(value=None, description='Start Date')

Box(children=(Button(icon='chevron-left', style=ButtonStyle()), IntSlider(value=0, max=9, readout=False), Butt…